In [1]:
city_code = "01450"
size = 100
api_root = "https://koumoul.com/data-fair/api/v1/datasets/dpe-france/lines"
url_api = (
    f"{api_root}?format=json&q_mode=simple&qs=code_insee_commune_actualise"
    + "%3A%22"
    + f"{city_code}"
    + "%22"
    + f"&size={size}&select="
    + "%2A&sampling=neighbors"
)

import requests
import pandas as pd

req = requests.get(url_api)
wb = req.json()
print(req.content[:1000])

b'{\n  "total": 121,\n  "next": "https://koumoul.com/data-fair/api/v1/datasets/dpe-france/lines?format=json&q_mode=simple&qs=code_insee_commune_actualise%3A%2201450%22&size=100&select=*&sampling=neighbors&after=102719%2C912454",\n  "results": [\n    {\n      "classe_consommation_energie": "E",\n      "tr001_modele_dpe_type_libelle": "Vente",\n      "annee_construction": 1,\n      "_geopoint": "45.927488,5.230195",\n      "latitude": 45.927488,\n      "surface_thermique_lot": 106.87,\n      "_i": 2,\n      "tr002_type_batiment_description": "Maison Individuelle",\n      "geo_adresse": "Rue du Chateau 01800 Villieu-Loyes-Mollon",\n      "_rand": 959550,\n      "code_insee_commune_actualise": "01450",\n      "estimation_ges": 9,\n      "geo_score": 0.58,\n      "classe_estimation_ges": "B",\n      "nom_methode_dpe": "M\xc3\xa9thode Facture",\n      "tv016_departement_code": "01",\n      "consommation_energie": 286,\n      "date_etablissement_dpe": "2013-04-15",\n      "longitude": 5.23019

In [2]:
import requests
import pandas as pd
import geopandas as gpd


def get_dpe_from_url(url):

    req = requests.get(url)
    wb = req.json()
    df = pd.json_normalize(wb["results"])

    dpe = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326
    )
    dpe = dpe.dropna(subset=["longitude", "latitude"])

    return dpe


dpe = get_dpe_from_url(url_api)
print(dpe.head(2))

  classe_consommation_energie tr001_modele_dpe_type_libelle  \
0                           E                         Vente   
1                           G                         Vente   

   annee_construction           _geopoint   latitude  surface_thermique_lot  \
0                   1  45.927488,5.230195  45.927488                 106.87   
1                1960  45.931376,5.230461  45.931376                  70.78   

   _i tr002_type_batiment_description  \
0   2             Maison Individuelle   
1   9             Maison Individuelle   

                                 geo_adresse   _rand  ...  \
0  Rue du Chateau 01800 Villieu-Loyes-Mollon  959550  ...   
1  552 Rue Royale 01800 Villieu-Loyes-Mollon  681070  ...   

  classe_estimation_ges  nom_methode_dpe  tv016_departement_code  \
0                     B  Méthode Facture                      01   
1                     D      Méthode 3CL                      01   

  consommation_energie date_etablissement_dpe longitude  _s

In [5]:
import seaborn as sns
import folium

palette = sns.color_palette("coolwarm", 8)


def interactive_map_dpe(dpe):

    # convert to number
    dpe["color"] = [
        ord(dpe.iloc[i]["classe_consommation_energie"].lower()) - 96
        for i in range(len(dpe))
    ]
    dpe = dpe.loc[dpe["color"] <= 7]
    dpe["color"] = [palette.as_hex()[x] for x in dpe["color"]]

    center = dpe[["latitude", "longitude"]].mean().values.tolist()
    sw = dpe[["latitude", "longitude"]].min().values.tolist()
    ne = dpe[["latitude", "longitude"]].max().values.tolist()

    m = folium.Map(location=center, tiles="OpenStreetMap")

    # I can add markers one by one on the map
    for i in range(0, len(dpe)):
        folium.Marker(
            [dpe.iloc[i]["latitude"], dpe.iloc[i]["longitude"]],
            popup=f"Year of construction: {dpe.iloc[i]['annee_construction']}, <br>DPE: {dpe.iloc[i]['classe_consommation_energie']}",
            icon=folium.Icon(
                color="black", icon="home", icon_color=dpe.iloc[i]["color"]
            ),
        ).add_to(m)

    m.fit_bounds([sw, ne])

    return m

m = interactive_map_dpe(dpe)
m

/opt/conda/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


API DVF

In [ ]:
url_api = "http://api.cquest.org/dvf?code_commune=29168"

req = requests.get(url_api)
wb = req.json()
print(wb.keys())
print(wb["nb_resultats"])

dict_keys(['source', 'derniere_maj', 'licence', 'nb_resultats', 'resultats'])
439


In [16]:
#Transactions sur des maisons

type_local = "Maison"
api_root = "http://api.cquest.org/dvf?code_commune=29168"
url_api = api_root + "&type_local=" + type_local

req = requests.get(url_api)
wb = req.json()
print(wb.keys())
print(wb["nb_resultats"])
df = pd.json_normalize(wb["resultats"])
df.head(2)

dict_keys(['source', 'derniere_maj', 'licence', 'nb_resultats', 'resultats'])
134


,code_service_ch,reference_document,articles_1,articles_2,articles_3,articles_4,articles_5,numero_disposition,date_mutation,nature_mutation,...,identifiant_local,surface_relle_bati,nombre_pieces_principales,nature_culture,nature_culture_speciale,surface_terrain,lat,lon,geom.type,geom.coordinates
0,None,None,None,None,None,None,None,000001,2015-06-25,Vente,...,None,90,4,S,None,277,48.042047,-4.705626,Point,"[-4.705626, 48.042047]"
1,None,None,None,None,None,None,None,000001,2015-09-12,Vente,...,None,90,4,S,None,615,48.038356,-4.709215,Point,"[-4.709215, 48.038356]"


In [ ]:
# Découpage communal de la ville de Plogoff
code = "29168"
url = "https://geo.api.gouv.fr/communes"
url_api = url + "?code=" + code

req = requests.get(url_api)
wb = req.json()
print(wb[0].keys())

dict_keys(['nom', 'code', 'codeDepartement', 'siren', 'codeEpci', 'codeRegion', 'codesPostaux', 'population'])


In [25]:
!pip install xlrd

In [26]:
# Permis

import pandas as pd
import xlrd
import geopandas as gpd

df = pd.read_excel(
    "https://www.data.gouv.fr/fr/datasets/r/d4b6b072-8a7d-4e04-a029-8cdbdbaf36a5",
    header=[0, 1],
)

# The Excel file has nested column names,
# we clean it
index_0 = [
    "" if df.columns[i][0].startswith("Unnamed") else df.columns[i][0]
    for i in range(len(df.columns))
]
index_1 = [df.columns[i][1] for i in range(len(df.columns))]
keep_index = [True if el in ("", "B") else False for el in index_0]
cols = [index_0[i] + " " + index_1[i].replace("+", "_") for i in range(len(df.columns))]
df.columns = cols
df = df.loc[:, keep_index]
df.columns = df.columns.str.replace("(^ |°)", "", regex=True).str.replace(" ", "_")

# We keep the subsample of interest
df = df.dropna(subset=["B_NB"])
df = df.loc[~df["B_NB"].astype(str).str.contains(r"(\%|\.)", regex=True), :]
df["B_NB"] = df["B_NB"].astype(int)
df["B_TR"] = (
    df["B_TR"].str.replace(",", ".").str.replace("%", "", regex=True).astype(float)
)
df = df.loc[df["B_NB"] > 20]

df.loc[:, ["Adresse", "CP", "Ville"]].head(5)

/tmp/ipykernel_11938/2250529706.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df.loc[~df["B_NB"].astype(str).str.contains(r"(\%|\.)", regex=True), :]


,Adresse,CP,Ville
0,56 RUE CHARLES ROBIN,01000,BOURG EN BRESSE
2,"7, avenue Revermont",01250,Ceyzeriat
3,72 PLACE DE LA MAIRIE,01000,SAINT-DENIS LES BOURG
4,6 RUE DU LYCEE,01000,BOURG EN BRESSE
5,9 place Edgard Quinet,01000,BOURG EN BRESSE


In [28]:
# API BAN

import time

dfgeoloc = df.loc[:, ["Adresse", "CP", "Ville"]].apply(
    lambda s: s.str.lower().str.replace(",", " ")
)
dfgeoloc["url"] = (
    dfgeoloc["Adresse"] + "+" + dfgeoloc["Ville"].str.replace("-", "+")
).str.replace(" ", "+")
dfgeoloc["url"] = (
    "https://api-adresse.data.gouv.fr/search/?q="
    + dfgeoloc["url"]
    + "&postcode="
    + df["CP"]
    + "&limit=1"
)
dfgeoloc = dfgeoloc.dropna()

start_time = time.time()


def get_geoloc(i):
    print(i)
    return gpd.GeoDataFrame.from_features(
        requests.get(dfgeoloc["url"].iloc[i]).json()["features"]
    )


local = [get_geoloc(i) for i in range(len(dfgeoloc.head(100)))]
print("--- %s seconds ---" % (time.time() - start_time))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
--- 7.340635538101196 seconds ---
